<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/pracaInzynierska.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.2 MB/s eta 0:00:00


In [3]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import csv

In [4]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
path='/content/drive/MyDrive/dyplom/EEG'

In [ ]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    for filename, label in zip(os.listdir(path), labels['PD']):
        if filename.endswith('.edf'):
            filepath = os.path.join(path, filename)
            raw = mne.io.read_raw_edf('PD129.edf', preload=True)
            annotations = mne.read_annotations('PD129.edf')
            raw.set_annotations(annotations)
            events = mne.make_fixed_length_events(raw, duration=1.0)
            epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1, reject_by_annotation='break')
            epochs_array.append([epochs, label])
            del epochs
            del raw
    return epochs_array


In [10]:
raw = mne.io.read_raw_edf('PD129.edf', preload=True)
annotations = mne.read_annotations('PD129.edf')
raw.set_annotations(annotations)
events = mne.make_fixed_length_events(raw, duration=1.0)
epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1, reject_by_annotation='break')
print(annotations.description)

Extracting EDF parameters from /content/PD129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 755199  =      0.000 ...  1474.998 secs...
Not setting metadata
1475 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
['Beginning of Recording' 'Montaz:PODLUZNY, Ref Channel Status Set '
 'Start Rejestracja' 'Recording Analyzer - QEEG'
 'Recording Analyzer - aEEG' 'Recording Analyzer - XLSpike'
 'Recording Analyzer - XLEvent / ECG' 'Recording Analyzer - CSA'
 'eyes closed' 'break' 'Break start' 'Impedance at 10 kOhm' 'Break end'
 'Montaz:Channel Test Referential' 'Montaz:PODLUZNY' 'break' 'Break start'
 'Impedance at 10 kOhm' 'Break end' 'break' 'Stop Rejestracja'
 'Break start' 'Montaz:PODLUZNY, Ref Channel Status Set '
 'Start Rejestracja' 'Break end' 'break' 'Break start'
 'Impedance at 10 kOhm' 'Break end' 'Montaz:Channel Test Referential'
 'Montaz:PODLUZNY' 'Montaz:UNIPOLARNY' 'Montaz:PODLU

In [ ]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=8, fmax=13, picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        for p in psds:
            data.append([p, label])
        del psds
        del freqs
    return data

In [ ]:
data = load_epochs_from_dir(path,labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

In [ ]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split([x[0] for x in psd], [x[1] for x in psd], test_size=0.2, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()
svm = SVC(kernel='linear', C=1, random_state=42)

In [ ]:
print(X_test_2d.shape)
print(X_train_2d.shape)

In [ ]:
svm.fit(X_train_2d, y_train)

In [ ]:
y_pred = svm.predict(X_test_2d)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
